In [2]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import time
import torch
from agent import DQNAgent
import pylab as plt0
# %matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import hockey.hockey_env as h_env
print("hi")

hi


## Helper Functions

In [3]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [4]:
# class DiscreteActionWrapper(gym.ActionWrapper):
#     def __init__(self, env: gym.Env, bins = 8):
#         """A wrapper for converting a 1D continuous actions into discrete ones.
#         Args:
#             env: The environment to apply the wrapper
#             bins: number of discrete actions
#         """
#         assert isinstance(env.action_space, spaces.Box)
#         super().__init__(env)
#         self.bins = bins
#         self.orig_action_space = env.action_space
#         self.action_space = spaces.Discrete(self.bins)


#     def action(self, action):
#         """ discrete actions from low to high in 'bins'
#         Args:
#             action: The discrete action
#         Returns:
#             continuous action
#         """
#         # return self.orig_action_space.low + action/(self.bins-1.0)*(self.orig_action_space.high-self.orig_action_space.low)
#         return h_env.HockeyEnv.discrete_to_continous_action(self, action)

## Test in Env

In [5]:
# env_name = 'Pendulum-v1'
# # env_name = 'CartPole-v0'
# env = gym.make(env_name)

env = h_env.HockeyEnv()
env.discretize_actions(8)

# if isinstance(env.action_space, spaces.Box):
#     print("Discretizing actions")
#     env = DiscreteActionWrapper(env)

ac_space = env.action_space
o_space = env.observation_space

In [6]:
print(ac_space)
print(o_space)

Discrete(8)
Box(-inf, inf, (18,), float32)


In [10]:
q_agent = DQNAgent(o_space, ac_space, {'env': 'hockey', 'numdiscreteactions': 8, 'double': False, 'per': False, 'dueling': False, 'rnd': False, 'multistep': None, 'gamma': 0.95, 'alpha': 0.002, 'epsilon': 0.3, 'epsilondecay': 1, 'minepsilon': 0.001, 'buffersize': 100000, 'batchsize': 128, 'hiddensize': 128, 'activation': 'ReLU', 'numseeds': 10, 'numepisodes': 20000, 'numsteps': 500, 'fititerations': 32, 'update_Qt_after': 20, 'opponent': 'random', 'curriculum': False, 'save': True, 'savepath': '../saved/agent.pk', 'test': True, 'plot': True, 'plotpath': '../plots/', 'verbose': False})

Train the agent!

In [12]:
stats = []
losses = []

max_episodes=1500
max_steps=500 
for i in range(max_episodes):
    # print("Starting a new episode")
    total_reward = 0
    ob, _info = env.reset()
    for t in range(max_steps):      # fill buffer
        done = False        
        qa = q_agent.act(ob)
        a1 = env.action(qa)
        a2 = np.random.uniform(-1,1,4)
        (ob_new, reward, done, trunc, info) = env.step(np.hstack([a1,a2]))
        total_reward+= reward
        q_agent.store_transition((ob, qa, reward, ob_new, done))            
        ob=ob_new        
        if done: 
            break    
    losses.extend(q_agent.train())
    stats.append([i,total_reward,t+1])    
    
    if ((i-1)%100==0):
        print("{}: Done after {} steps. Reward: {}".format(i, t+1, total_reward))

1: Done after 251 steps. Reward: 0.0
101: Done after 251 steps. Reward: 0.0
201: Done after 251 steps. Reward: 0.0
301: Done after 196 steps. Reward: 1096.7881363071258
401: Done after 251 steps. Reward: 0.0
501: Done after 89 steps. Reward: 1091.522923936868
601: Done after 102 steps. Reward: 1030.7763320330369
701: Done after 251 steps. Reward: -85.10210583533573
801: Done after 251 steps. Reward: 0.0
901: Done after 251 steps. Reward: 0.0
1001: Done after 251 steps. Reward: 0.0
1101: Done after 251 steps. Reward: 0.0
1201: Done after 143 steps. Reward: 956.752115982391
1301: Done after 251 steps. Reward: -25.08521387006572
1401: Done after 251 steps. Reward: 0.0


Plot the training reward over time. Use the running_mean(array, window_size) to plot a smooth version 

In [13]:
stats_np = np.asarray(stats)
fig=plt.figure(figsize=(6,3.8))
plt.plot(stats_np[:,1], label="return")
plt.plot(running_mean(stats_np[:,1],100), label="smoothed-return")
plt.legend()

NameError: name 'plt' is not defined

In [14]:
losses_np = np.asarray(losses)
plt.figure(figsize=(45,3))
plt.plot(losses_np)

NameError: name 'plt' is not defined

## Evaluate (without exploration)

Please look at the behavior for a small number of episodes

In [20]:
env = h_env.HockeyEnv()
env.discretize_actions(8)
player2 = h_env.BasicOpponent(weak=True)

In [21]:
test_stats = []
wins = []
episodes=10

for i in range(episodes):
    total_reward = 0
    ob, _info = env.reset()
    obs_agent2 = env.obs_agent_two()
    for t in range(max_steps):
        env.render()
        done = False        
        a1_discrete = q_agent.act(ob)
        a1 = env.action(a1_discrete)
        a2 = player2.act(obs_agent2) # np.random.uniform(-1,1,4) #
        (ob_new, reward, done, trunc, _info) = env.step(np.hstack([a1,a2]))
        total_reward+= reward
        ob=ob_new
        obs_agent2 = env.obs_agent_two()        
        if done: break    
    test_stats.append([i,total_reward,t+1]) 
    wins.append(_info['winner'])       

Evaluate mean and standard deviation of performance 

(for the Pendulum: an average return around -30 or better should be achieved)

(for the CartPendulum it is possible to get 200)

In [ ]:
test_stats_np = np.array(test_stats)
print(np.mean(test_stats_np[:,1]), "+-", np.std(test_stats_np[:,1]))

# Visualize

Visualization of the value function.

In [ ]:
Adapt the value_function plotting from last time to plot the maxQ value

## Pendulum Env

Observation space:

0 angle

1 angular velocity

Do that for the pendulum function. Does it look like you expect?

Do that for the pendulum function. Does it look like you expect

## Cartpole Env

Observation space:
 
0       Cart Position             -4.8                    4.8

1       Cart Velocity             -Inf                    Inf

2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)

3       Pole Angular Velocity     -Inf                    Inf

Try to adapt the plotting function that it also works in higher input spaces where all other inputs are 0

In [ ]:
figQ = plot_Q_function_generic(q_agent.Q, input_dims=o_space.shape[0], plot_dim1=0, plot_dim2=2, 
                       label_dim1="Cart Pos", label_dim2="Pole Angle")

In [ ]:
figQ = plot_Q_function_generic(q_agent.Q, input_dims=o_space.shape[0], plot_dim1=0, plot_dim2=1, 
                       label_dim1="Cart Pos", label_dim2="Cart Vel")

In [ ]:
figQ = plot_Q_function_generic(q_agent.Q, input_dims=o_space.shape[0], plot_dim1=2, plot_dim2=3, 
                       label_dim1="Pol Angle", label_dim2="Pole Vel")

In [ ]:
# env_name = 'Acrobot-v1'
# env_name = 'MountainCar-v0'
# env_name = 'LunarLander-v2'